**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install scikit-video

     |████████████████████████████████| 2.3MB 38.8MB/s 


In [0]:
!pip install opencv-python

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from collections import deque

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.regularizers import l2
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization

%tensorflow_version 1.x

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon  = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e):
        self.epsilon = e

    def act(self, s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The act function allows to select an action to be performed in the environnement. The choice of the action depends on the phase. 
In the training phase, we perform a random action from $A = \{0,1,2,3\}$ with an $\epsilon$ probability to encourage exploration, or perform an action according to the learned policy with a $1-\epsilon$ probability to exploit the current policy. 
In the test phase, the agent performs the best action with respect to the current state according to the learned policy.

The choice of the parameter $\epsilon$ is crucial as the larger is its value, the more it allows the agent to explore the environnement. It encourages the agent to avoid getting stuck in local reward optimas.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size   = grid_size
        self.max_time    = max_time
        self.temperature = temperature

        #board on which one plays
        self.board    = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0
        self.scale = 16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self, e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size,self.grid_size,3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x, self.y, :] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :]  = 0
        b[:, :2, :]  = 0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:] = b

    def act(self, action):
        """This function returns the new state, reward and decides if the game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.position[self.x, self.y] = 1

        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)), axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size - 3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size - 3, size=1)[0]

        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x, self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)), axis=2)

        state = state[self.x - 2: self.x + 3, self.y - 2 : self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T    = 200
temperature  = 0.3
epochs_train = 50 # set small when debugging
epochs_test  = 30 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array `board` defines the spatial distribution of the rewards on the  $N \times N$ grid. It stores the reward of getting to a state $(x,y)$ in the board, which is set to 0 after its collection, right after the state has already been visited. 

If we get a closer look at `get_frame`, which is a graphic representation of the environement, red cells contain positive rewards, blue cells correspond to negative rewards, the position of the rat is represented by a white cell and black cells correspond to borders.

The array `position` is a static array that defines the borders -- forbidden states -- of the grid by -1, the allowed states by 0 and the position of the agent by 1.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
    #loss = 0

    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the game will end
        game_over = False

        win  = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            # Apply an action to the environment, get the next state, the reward and the state of the game
            previous_state = state
            state, reward, game_over = env.act(action)
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
            # Apply the reinforcement strategy
            #loss = agent.reinforce(previous_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e+1))
        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Win/lose count {}/{} ({}) | Average score {:.3f}".format(e+1, epochs, str(win), str(lose), str(win-lose), score/(1+e)))
        agent.save()

    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random1.mp4'))

Epoch 001/030 | Win/lose count 15.5/15.0 (0.5) | Average score 0.500
Epoch 002/030 | Win/lose count 11.0/18.0 (-7.0) | Average score -3.250
Epoch 003/030 | Win/lose count 9.0/8.0 (1.0) | Average score -1.833
Epoch 004/030 | Win/lose count 10.5/12.0 (-1.5) | Average score -1.750
Epoch 005/030 | Win/lose count 3.5/12.0 (-8.5) | Average score -3.100
Epoch 006/030 | Win/lose count 9.5/15.0 (-5.5) | Average score -3.500
Epoch 007/030 | Win/lose count 6.5/13.0 (-6.5) | Average score -3.929
Epoch 008/030 | Win/lose count 11.5/12.0 (-0.5) | Average score -3.500
Epoch 009/030 | Win/lose count 13.0/12.0 (1.0) | Average score -3.000
Epoch 010/030 | Win/lose count 3.5/9.0 (-5.5) | Average score -3.250
Epoch 011/030 | Win/lose count 12.5/16.0 (-3.5) | Average score -3.273
Epoch 012/030 | Win/lose count 10.0/12.0 (-2.0) | Average score -3.167
Epoch 013/030 | Win/lose count 12.5/14.0 (-1.5) | Average score -3.038
Epoch 014/030 | Win/lose count 14.5/9.0 (5.5) | Average score -2.429
Epoch 015/030 | Win

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__1)__ Let $\pi$ be a policy.
$$
\begin{align*} 
Q^\pi(s,a)&=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=E_{p^{\pi}}[r(s_0,a_0) + \gamma\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=r(s,a) + \gamma \sum_{s',a'} p^\pi(s_1=s',a_1=a'|s_0=s,a_0=a)E_{p^{\pi}}[\sum_{t=1}^{\infty}\gamma^{t-1} r(s_{t},a_{t})|s_{1}=s',a_{1}=a'] \quad &\text{Markov Property}\\
          &=r(s,a) + \gamma \sum_{s',a'} p^\pi(s_1=s',a_1=a'|s_0=s,a_0=a)E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t} r(s_{t},a_{t})|s_{0}=s',a_{0}=a'] \quad &\text{MDP and change of time}\\
          &=r(s,a) + \gamma E_{(s',a')\sim p^\pi(.|s,a)}\left(E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          &=E_{(s',a')\sim p^\pi(.|s,a)}\left(r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          &=E_{(s',a')\sim p^\pi(.|s,a)}\left(r(s,a) + \gamma Q^{\pi}(s',a') \right).
\end{align*}
$$


__2)__ 
The optimal policy $\pi^*$ is deterministic verifying $\pi^*(s') = arg\max_{a'} Q^{*}(s',a')$ and $Q^*(s,a)=Q^{\pi^*}(s,a).$

We have $(s',a')\sim p^\pi(.|s,a)$, which breaks down then to $s'\sim p(.|s,a)$ and $a'\sim \pi(.|s')$.


\begin{align*}
Q^*(s,a)&= E_{(s',a')\sim p^{\pi^{*}}(.|s,a)}\left(r(s,a) + \gamma Q^{*}(s',a') \right)\\
        &= E_{s' \sim p(.|s,a),\, a'\sim \pi^*(.|s')}\left(r(s,a) + \gamma Q^{*}(s',a') \right)\\
         &= E_{s'\sim p(.|s,a),\, a' = \pi^*(s')}\left(r(s,a) + \gamma Q^{*}(s',a') \right) \quad \textrm{Deterministic policy}\\
        &= E_{s' \sim p(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*}(s',a') \right)\\
\end{align*}


__3)__
We have shown that $Q^*$ is the solution of the equation $F(Q) = 0$ with $$F(Q) = E_{s' \sim p(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q(s',a') - Q(s,a)\right).$$

Minimizing $\mathcal{L}(\theta)=E_{s' \sim p(.|s,a)}\Vert r(s,a) +\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$ will lead us to find the optimal Q function, noted $Q^*$, hence the optimal policy $\pi^*$.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = deque(maxlen = self.max_memory) 
        
    def remember(self, m):
        self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward and if the games end
            previous_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(previous_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})".format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5', name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon=0.1, memory_size=100, batch_size=16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        # number of state
        self.n_state = n_state
        # Memory
        self.memory = Memory(memory_size)
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None]))

    def reinforce(self, s_, next_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, next_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q     = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            # Take a random batch from our buffer
            s_, next_s_, a_, r_, game_over_ = self.memory.random_access()
            # Store the current state and generate its state-action function
            input_states[i], target_q[i]    = s_, self.model.predict(s_[None])
            
            #f For the action performed, set target q depending if the state is terminal or not
            if game_over_:
                target_q[i, a_] = r_
            else:
                ns_q = self.model.predict(next_s_[None])
                target_q[i, a_] = r_ + self.discount*np.max(ns_q, axis = 1)

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -5, 5)
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
         
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        # NN Model
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(units = 32, input_shape=(5*5*self.n_state, ), activation= 'relu', kernel_regularizer = l2(1e-4)))
        model.add(Dense(units = 4, input_shape=(5*5*self.n_state, ), kernel_regularizer = l2(1e-4)))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model   

In [0]:
env   = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon=.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))













Epoch 000/050 | Loss 0.0075 | Win/lose count 2.5/5.0 (-2.5)
Epoch 001/050 | Loss 0.0070 | Win/lose count 0.5/4.0 (-3.5)
Epoch 002/050 | Loss 0.0217 | Win/lose count 4.0/5.0 (-1.0)
Epoch 003/050 | Loss 0.0142 | Win/lose count 7.0/5.0 (2.0)
Epoch 004/050 | Loss 0.0215 | Win/lose count 6.0/3.0 (3.0)
Epoch 005/050 | Loss 0.0472 | Win/lose count 8.0/6.0 (2.0)
Epoch 006/050 | Loss 0.0272 | Win/lose count 0.5/5.0 (-4.5)
Epoch 007/050 | Loss 0.0092 | Win/lose count 3.0/1.0 (2.0)
Epoch 008/050 | Loss 0.0449 | Win/lose count 9.0/4.0 (5.0)
Epoch 009/050 | Loss 0.0392 | Win/lose count 1.5/1.0 (0.5)
Epoch 010/050 | Loss 0.0190 | Win/lose count 3.0/5.0 (-2.0)
Epoch 011/050 | Loss 0.0182 | Win/lose count 2.5/2.0 (0.5)
Epoch 012/050 | Loss 0.0164 | Win/lose count 3.0/3.0 (0.0)
Epoch 013/050 | Loss 0.0154 | Win/lose count 6.5/1.0 (5.5)
Epoch 014/050 | Loss 0.0128 | Win/lose count 6.5/8.0 (-1.5)
Epoch 015/050 | Loss 0.0183 | Win/lose count 4.0/3.0 (1.0)
Epoch 016/050 | Loss 0.0147 | Win/lose

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args, lr=0.1, **kwargs):
        super(DQN_CNN, self).__init__(*args, **kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, activation='relu', kernel_regularizer=l2(1e-4), input_shape=(5,5,self.n_state, )))
        model.add(Conv2D(64, kernel_size=3, activation='relu', kernel_regularizer=l2(1e-4)))
        model.add(Flatten())
        model.add(Dense(units=4, kernel_regularizer=l2(1e-4)))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0095 | Win/lose count 2.0/4.0 (-2.0)
Epoch 001/050 | Loss 0.0119 | Win/lose count 6.0/4.0 (2.0)
Epoch 002/050 | Loss 0.0098 | Win/lose count 1.5/5.0 (-3.5)
Epoch 003/050 | Loss 0.0254 | Win/lose count 3.5/1.0 (2.5)
Epoch 004/050 | Loss 0.0089 | Win/lose count 3.0/1.0 (2.0)
Epoch 005/050 | Loss 0.0269 | Win/lose count 4.5/1.0 (3.5)
Epoch 006/050 | Loss 0.0097 | Win/lose count 4.5/3.0 (1.5)
Epoch 007/050 | Loss 0.1814 | Win/lose count 6.0/1.0 (5.0)
Epoch 008/050 | Loss 0.0143 | Win/lose count 1.0/3.0 (-2.0)
Epoch 009/050 | Loss 0.0078 | Win/lose count 2.5/2.0 (0.5)
Epoch 010/050 | Loss 0.0085 | Win/lose count 3.5/2.0 (1.5)
Epoch 011/050 | Loss 0.0078 | Win/lose count 3.5/3.0 (0.5)
Epoch 012/050 | Loss 0.0075 | Win/lose count 5.5/2.0 (3.5)
Epoch 013/050 | Loss 0.0073 | Win/lose count 2.5/1.0 (1.5)
Epoch 014/050 | Loss 0.0084 | Win/lose count 5.5/3.0 (2.5)
Epoch 015/050 | Loss 0.0077 | Win/lose count 8.5/0 (8.5)
Epoch 016/050 | Loss 0.1581 | Win/lose count 5.5/2.0 (3

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)         # We have set temperature to many values within [0.1, 0.9]
agent_cnn = DQN_CNN(size, lr=.1, epsilon=.1, memory_size=2000, batch_size=32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon=.1, memory_size=2000, batch_size=32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix='cnn_test')
print('Test of the FC')
test(agent_fc, env, epochs_test, prefix='fc_test')

Test of the CNN
Epoch 001/030 | Win/lose count 4.0/2.0 (2.0) | Average score 2.000
Epoch 002/030 | Win/lose count 6.0/1.0 (5.0) | Average score 3.500
Epoch 003/030 | Win/lose count 8.5/3.0 (5.5) | Average score 4.167
Epoch 004/030 | Win/lose count 10.0/3.0 (7.0) | Average score 4.875
Epoch 005/030 | Win/lose count 10.5/2.0 (8.5) | Average score 5.600
Epoch 006/030 | Win/lose count 16.0/2.0 (14.0) | Average score 7.000
Epoch 007/030 | Win/lose count 4.5/3.0 (1.5) | Average score 6.214
Epoch 008/030 | Win/lose count 5.5/2.0 (3.5) | Average score 5.875
Epoch 009/030 | Win/lose count 4.5/2.0 (2.5) | Average score 5.500
Epoch 010/030 | Win/lose count 13.0/3.0 (10.0) | Average score 5.950
Epoch 011/030 | Win/lose count 6.0/5.0 (1.0) | Average score 5.500
Epoch 012/030 | Win/lose count 9.0/3.0 (6.0) | Average score 5.542
Epoch 013/030 | Win/lose count 5.0/1.0 (4.0) | Average score 5.423
Epoch 014/030 | Win/lose count 7.0/4.0 (3.0) | Average score 5.250
Epoch 015/030 | Win/lose count 7.5/6.0 (

In [0]:
HTML(display_videos('cnn_test21.mp4'))

In [0]:
HTML(display_videos('fc_test21.mp4'))

__Comments__

- We notice that the agent lacks of exploration of the grid and gets stuck when there is no positive reward within his reach.

- Increasing temperature increases the number of positive rewards cells, which leads the agent to explore more since positive reward is most of the time within his visibility. Thus, the final average score gets higher.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss  = 0
    
    #exponentially decreasing epsilon from init_eps to approximately 0.1 for each epoch
    epsilon_init  = agent.epsilon
    epsilon_final = 0.1
    T = epoch/5.0
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win     = 0
        lose    = 0
        epsilon = (epsilon_init - epsilon_final) * np.exp(-e/T) + epsilon_final
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward and if the games end
            previous_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win  = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            agent.set_epsilon(epsilon)
            loss = agent.reinforce(previous_state, state, action, reward, game_over)
      
        # Save as a mp4
        if (e + 1) % 10 == 0:
            env.draw(prefix+str(e+1))

        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.3f}/{:.3f} ({:.3f}), epsilon {:.3f}".format(e, epoch, loss, win, lose, win-lose, epsilon))
      
    agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
        
class EnvironmentExploring(object):
  
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size   = grid_size
        self.max_time    = max_time
        self.temperature = temperature

        #board on which one plays
        self.board    = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale   = 16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x, self.y, :] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :]  = 0
        b[:, :2, :]  = 0
        
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t,:,:,:] = b

    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        reward = self.board[self.x, self.y] - train * self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [0]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon=.9, memory_size=2000, batch_size=32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0213 | Win/lose count 6.500/29.100 (-22.600), epsilon 0.900
Epoch 001/050 | Loss 0.0092 | Win/lose count 9.000/19.800 (-10.800), epsilon 0.824
Epoch 002/050 | Loss 0.0141 | Win/lose count 10.500/23.500 (-13.000), epsilon 0.755
Epoch 003/050 | Loss 0.0162 | Win/lose count 14.000/23.600 (-9.600), epsilon 0.693
Epoch 004/050 | Loss 0.0193 | Win/lose count 14.500/27.000 (-12.500), epsilon 0.636
Epoch 005/050 | Loss 0.0168 | Win/lose count 15.500/25.200 (-9.700), epsilon 0.585
Epoch 006/050 | Loss 0.0157 | Win/lose count 18.500/19.000 (-0.500), epsilon 0.539
Epoch 007/050 | Loss 0.1616 | Win/lose count 13.500/23.600 (-10.100), epsilon 0.497
Epoch 008/050 | Loss 0.0144 | Win/lose count 17.500/26.000 (-8.500), epsilon 0.459
Epoch 009/050 | Loss 0.0111 | Win/lose count 16.500/25.500 (-9.000), epsilon 0.425
Epoch 010/050 | Loss 0.0147 | Win/lose count 19.500/15.600 (3.900), epsilon 0.394
Epoch 011/050 | Loss 0.0159 | Win/lose count 20.000/23.100 (-3.100), epsilon 0.366
Ep

In [0]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore21.mp4'))

Epoch 001/030 | Win/lose count 18.0/5.0 (13.0) | Average score 13.000
Epoch 002/030 | Win/lose count 15.0/5.0 (10.0) | Average score 11.500
Epoch 003/030 | Win/lose count 25.5/3.0 (22.5) | Average score 15.167
Epoch 004/030 | Win/lose count 13.5/3.0 (10.5) | Average score 14.000
Epoch 005/030 | Win/lose count 18.0/6.0 (12.0) | Average score 13.600
Epoch 006/030 | Win/lose count 16.5/4.0 (12.5) | Average score 13.417
Epoch 007/030 | Win/lose count 13.5/3.0 (10.5) | Average score 13.000
Epoch 008/030 | Win/lose count 14.5/4.0 (10.5) | Average score 12.688
Epoch 009/030 | Win/lose count 24.5/4.0 (20.5) | Average score 13.556
Epoch 010/030 | Win/lose count 18.5/6.0 (12.5) | Average score 13.450
Epoch 011/030 | Win/lose count 11.5/6.0 (5.5) | Average score 12.727
Epoch 012/030 | Win/lose count 19.0/9.0 (10.0) | Average score 12.500
Epoch 013/030 | Win/lose count 18.5/2.0 (16.5) | Average score 12.808
Epoch 014/030 | Win/lose count 12.5/5.0 (7.5) | Average score 12.429
Epoch 015/030 | Win/lo

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***